# Defining parameters

In [5]:
import pandas as pd, numpy as np, functools, scipy, xarray as xr
import toolbox, tqdm, pathlib
from typing import List, Tuple, Dict, Any, Literal
import matplotlib.pyplot as plt
import matplotlib as mpl, seaborn as sns
from scipy.stats import gaussian_kde
import math, itertools, pickle, logging, beautifullogger
from xarray_helper import apply_file_func, auto_remove_dim, nunique, apply_file_func_decorator, extract_unique, mk_bins
import scipy.signal
from matplotlib.backends.backend_pdf import PdfPages

xr.set_options(use_flox=True, display_expand_coords=True, display_max_rows=100, display_expand_data_vars=True, display_width=150)
logger = logging.getLogger(__name__)
beautifullogger.setup(displayLevel=logging.INFO)
logging.getLogger("flox").setLevel(logging.WARNING)
tqdm.tqdm.pandas(desc="Computing")

MODE: Literal["TEST", "ALL", "SMALL", "BALANCED"]="ALL"
FIGS = ["coherence_phase_figs_structure"] #["pwelch_mean_figs_structure", "pwelch_mean_figs_species", "coherence_mean_figs_structure"]
DISPLAY=True

match MODE:
    case "TEST":
        cache_path = "/media/julien/data1/JulienCache/Test/"
    case "ALL" | "BALANCED":
        cache_path = f"/media/julien/data1/JulienCache/{'All' if MODE=='ALL' else 'Balanced'}/"
    case "SMALL":
        cache_path = "/media/julien/data1/JulienCache/Small/"


group_cols = ["Species", "Structure", "Healthy"]
pair_group_cols = [x+"_1" for x in group_cols] + [x+"_2" for x in group_cols]
species_order = ["Rat", "Monkey", "Human"]
structure_order = ["GPe", "STN", "STR"]
condition_order = [0, 1]
sig_type_order=["bua", "lfp", "spike_times"]

# Loading dataset

In [6]:
signals: xr.Dataset = pickle.load(open(cache_path+"signals_computed.pkl", "rb"))
signal_pairs: xr.Dataset = pickle.load(open(cache_path+"signal_pairs_computed.pkl", "rb"))
signal_pairs = signal_pairs.where((signal_pairs["FullStructure_1"] != "STN_VMNR") & (signal_pairs["FullStructure_2"] != "STN_VMNR"), drop=True)
signals

<xarray.Dataset>
Dimensions:                    (Contact: 5456, sig_preprocessing: 5, f: 46, f2: 48)
Coordinates:
    Condition                  (Contact) object 'Park' 'Control_vnmr' 'Park' 'Control_vnmr' 'Control_vnmr' ... '6ohda' '6ohda' '6ohda' '6ohda' '6ohda'
    CorticalState              (Contact) object 'Awake' 'Awake' 'Awake' 'Awake' ... 'Anesthetized' 'Anesthetized' 'Anesthetized' 'Anesthetized'
    Session                    (Contact) object 'HS#1' 'HS#100' 'HS#100' 'HS#1000' 'HS#1002' ... 'RSa03#17' 'RSa03#17' 'RSa03#17' 'RSa03#17'
    FullStructure              (Contact) object 'STN_DLOR' 'STN_VMNR' 'STN_DLOR' 'STN_VMNR' 'STN_VMNR' 'STN_VMNR' ... 'STR' 'STR' 'STR' 'STR' 'STR'
    Subject                    (Contact) object 'Unknown' 'Unknown' 'Unknown' 'Unknown' 'Unknown' 'Unknown' ... 'L28' 'L28' 'L28' 'L28' 'L28' 'L28'
    Species                    (Contact) object 'Human' 'Human' 'Human' 'Human' 'Human' 'Human' 'Human' ... 'Rat' 'Rat' 'Rat' 'Rat' 'Rat' 'Rat' 'Rat'
    has_entry                  (Contact, sig_preprocessing) bool True False True False False True False ... True False True True True False False
    Structure                  (Contact) object 'STN' 'STN' 'STN' 'STN' 'STN' 'STN' 'STN' 'STN' ... 'STR' 'STR' 'STR' 'STR' 'STR' 'STR' 'STR' 'STR'
  * Contact                    (Contact) int32 0 1 2 3 4 5 6 7 8 9 10 11 12 13 ... 5455 5456 5457 5458 5459 5460 5461 5462 5463 5464 5465 5466 5467
    Healthy                    (Contact) int8 0 1 0 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
    Date                       (Contact) datetime64[ns] 2009-01-14 2009-01-29 2009-01-29 2009-06-24 ... 2007-03-09 2007-03-09 2007-03-09 2007-03-09
    sig_type                   (sig_preprocessing) object 'bua' 'lfp' 'spike_times' 'spike_times' 'spike_times'
  * sig_preprocessing          (sig_preprocessing) object 'bua' 'lfp' 'neuron_0' 'neuron_1' 'neuron_2'
    group_index                (Contact) object ('Human', 'STN', 0) ('Human', 'STN', 1) ('Human', 'STN', 0) ... ('Rat', 'STR', 0) ('Rat', 'STR', 0)
  * f                          (f) float64 48.4 45.68 43.12 40.7 38.42 36.26 34.22 32.3 30.49 ... 5.711 5.39 5.087 4.802 4.532 4.278 4.038 3.811 3.597
  * f2                         (f2) float64 3.0 4.0 5.0 6.0 7.0 8.0 9.0 10.0 11.0 12.0 13.0 ... 40.0 41.0 42.0 43.0 44.0 45.0 46.0 47.0 48.0 49.0 50.0
Data variables:
    time_representation_path   (Contact, sig_preprocessing) object 'time_repr/Species=Human/Structure=STN_DLOR/signal_resampled_type=bua/Healthy=Fa...
    duration                   (sig_preprocessing, Contact) float64 45.87 9.772 9.756 22.87 15.14 11.01 8.44 9.97 ... nan nan nan nan nan nan nan nan
    start_time                 (sig_preprocessing, Contact) float64 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 ... nan nan nan nan nan nan nan nan nan
    end_time                   (sig_preprocessing, Contact) float64 45.87 9.772 9.756 22.87 15.14 11.01 8.44 9.97 ... nan nan nan nan nan nan nan nan
    resampled_continuous_path  (Contact, sig_preprocessing) object '/media/julien/data1/JulienCache/All/resampled_normalized/time_repr/Species=Huma...
    cwt_path                   (Contact, sig_preprocessing) object '/media/julien/data1/JulienCache/All/cwt/resampled_normalized/time_repr/Species=...
    cwt_power                  (Contact, sig_preprocessing) object '/media/julien/data1/JulienCache/All/power_cwt/cwt/resampled_normalized/time_rep...
    spectrogram                (Contact, sig_preprocessing) object '/media/julien/data1/JulienCache/All/spectrogram/resampled_normalized/time_repr/...
    pwelch_cwt                 (Contact, sig_preprocessing, f) float64 0.01935 0.03201 0.05407 0.08765 0.1343 0.1926 0.255 ... nan nan nan nan nan nan
    pwelch_spectrogram         (Contact, sig_preprocessing, f2) float64 0.008827 0.005515 0.005699 0.006159 0.005872 0.0048 ... nan nan nan nan nan

# Add group coordinates + diverse stuff to dataset

In [7]:
dataset = xr.merge([signals, signal_pairs])
dataset = dataset[[var for var in dataset.variables if ("pwelch" in var) or "coherence" in var or "duration" in var]]
scipy_freq_coords = dataset["f2"].to_numpy()
dataset = dataset.interp(f=np.linspace(3, 50, 94, endpoint=False), f2=np.linspace(3, 50, 94, endpoint=False))
for col in dataset.variables:
    if "f2" in dataset[col].dims:
        dataset[col] = dataset[col].rename(f2="f_interp")
    if "f" in dataset[col].dims:
        dataset[col] = dataset[col].rename(f="f_interp")
dataset = dataset.drop(["f", "f2"])
dataset["is_scipy_freq"] = dataset["f_interp"].isin(np.round(scipy_freq_coords*2)/2)
dataset = dataset.set_coords("is_scipy_freq")
dataset["pwelch"] = dataset[["pwelch_cwt", "pwelch_spectrogram"]].to_array(dim="spectral_analysis_function")
dataset.drop_vars(["pwelch_cwt", "pwelch_spectrogram"])
dataset["spectral_analysis_function"] = dataset["spectral_analysis_function"].str.replace("pwelch_", "").str.replace("spectrogram", "scipy.spectrogram").str.replace("cwt", "pycwt.cwt")
coherence = dataset[["coherence_scipy"]].to_array(dim="spectral_analysis_function", name="coherence")
coherence["spectral_analysis_function"] = coherence["spectral_analysis_function"].str.replace("coherence_", "").str.replace("scipy", "scipy.coherence").str.replace("wct", "pycwt.wct")
dataset=xr.merge([dataset, coherence])
dataset = dataset.drop_vars(["coherence_scipy"])



for dim, groupcols in dict(Contact=group_cols, Contact_pair=pair_group_cols, sig_preprocessing=["sig_type"], sig_preprocessing_pair=["sig_type_1", "sig_type_2"]).items():
    grpname = dim+"_grp"
    dataset[grpname] = xr.DataArray(
        pd.MultiIndex.from_arrays([dataset[a].data for a in groupcols],names=groupcols), 
        dims=[dim], coords=[dataset[dim]]
    )
    dataset = dataset.set_coords(grpname)

dataset["coherence_norm"] = np.abs(dataset["coherence"])
dataset

<xarray.Dataset>
Dimensions:                     (Contact: 5456, sig_preprocessing: 5, sig_preprocessing_pair: 15, Contact_pair: 27559, f_interp: 94,
                                 spectral_analysis_function: 3)
Coordinates:
  * Contact                     (Contact) int32 0 1 2 3 4 5 6 7 8 9 10 11 12 13 ... 5455 5456 5457 5458 5459 5460 5461 5462 5463 5464 5465 5466 5467
  * sig_preprocessing           (sig_preprocessing) object 'bua' 'lfp' 'neuron_0' 'neuron_1' 'neuron_2'
  * sig_preprocessing_pair      (sig_preprocessing_pair) object MultiIndex
  * sig_preprocessing_1         (sig_preprocessing_pair) object 'bua' 'bua' 'bua' 'bua' 'bua' ... 'neuron_0' 'neuron_1' 'neuron_1' 'neuron_2'
  * sig_preprocessing_2         (sig_preprocessing_pair) object 'bua' 'lfp' 'neuron_0' 'neuron_1' ... 'neuron_2' 'neuron_1' 'neuron_2' 'neuron_2'
  * Contact_pair                (Contact_pair) object MultiIndex
  * Contact_1                   (Contact_pair) int32 139 140 144 145 147 148 150 151 157 158 163 ... 5467 5467 5467 5467 5467 5467 5467 5467 5467 5467
  * Contact_2                   (Contact_pair) int32 140 139 145 144 148 147 151 150 158 157 164 ... 5457 5458 5459 5460 5461 5462 5463 5464 5465 5466
  * f_interp                    (f_interp) float64 3.0 3.5 4.0 4.5 5.0 5.5 6.0 6.5 7.0 7.5 8.0 ... 45.0 45.5 46.0 46.5 47.0 47.5 48.0 48.5 49.0 49.5
  * spectral_analysis_function  (spectral_analysis_function) object 'pycwt.cwt' 'scipy.coherence' 'scipy.spectrogram'
    Condition                   (Contact) object 'Park' 'Control_vnmr' 'Park' 'Control_vnmr' 'Control_vnmr' ... '6ohda' '6ohda' '6ohda' '6ohda'
    CorticalState               (Contact) object 'Awake' 'Awake' 'Awake' 'Awake' ... 'Anesthetized' 'Anesthetized' 'Anesthetized' 'Anesthetized'
    Session                     (Contact) object 'HS#1' 'HS#100' 'HS#100' 'HS#1000' 'HS#1002' ... 'RSa03#17' 'RSa03#17' 'RSa03#17' 'RSa03#17'
    FullStructure               (Contact) object 'STN_DLOR' 'STN_VMNR' 'STN_DLOR' 'STN_VMNR' 'STN_VMNR' 'STN_VMNR' ... 'STR' 'STR' 'STR' 'STR' 'STR'
    Subject                     (Contact) object 'Unknown' 'Unknown' 'Unknown' 'Unknown' 'Unknown' 'Unknown' ... 'L28' 'L28' 'L28' 'L28' 'L28' 'L28'
    Species                     (Contact) object 'Human' 'Human' 'Human' 'Human' 'Human' 'Human' 'Human' ... 'Rat' 'Rat' 'Rat' 'Rat' 'Rat' 'Rat' 'Rat'
    has_entry                   (Contact, sig_preprocessing) bool True False True False False True False ... True False True True True False False
    Structure                   (Contact) object 'STN' 'STN' 'STN' 'STN' 'STN' 'STN' 'STN' 'STN' ... 'STR' 'STR' 'STR' 'STR' 'STR' 'STR' 'STR' 'STR'
    Healthy                     (Contact) int8 0 1 0 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
    Date                        (Contact) datetime64[ns] 2009-01-14 2009-01-29 2009-01-29 2009-06-24 ... 2007-03-09 2007-03-09 2007-03-09 2007-03-09
    sig_type                    (sig_preprocessing) object 'bua' 'lfp' 'spike_times' 'spike_times' 'spike_times'
    group_index                 (Contact) object ('Human', 'STN', 0) ('Human', 'STN', 1) ('Human', 'STN', 0) ... ('Rat', 'STR', 0) ('Rat', 'STR', 0)
    Condition_1                 (Contact_pair) object 'Park' 'Park' 'Park' 'Park' 'Park' 'Park' ... '6ohda' '6ohda' '6ohda' '6ohda' '6ohda' '6ohda'
    CorticalState_1             (Contact_pair) object 'Awake' 'Awake' 'Awake' 'Awake' ... 'Anesthetized' 'Anesthetized' 'Anesthetized' 'Anesthetized'
    Session_1                   (Contact_pair) object 'HS#1246' 'HS#1246' 'HS#1250' 'HS#1250' ... 'RSa03#17' 'RSa03#17' 'RSa03#17' 'RSa03#17'
    FullStructure_1             (Contact_pair) object 'STN_DLOR' 'STN_DLOR' 'STN_DLOR' 'STN_DLOR' 'STN_DLOR' ... 'STR' 'STR' 'STR' 'STR' 'STR'
    Subject_1                   (Contact_pair) object 'Unknown' 'Unknown' 'Unknown' 'Unknown' 'Unknown' 'Unknown' ... 'L28' 'L28' 'L28' 'L28' 'L28'
    Species_1                   (Contact_pair) object 

# Creating dataset_contact

In [8]:
dataset_contact = xr.Dataset()
dataset_contact["pwelch"] = dataset["pwelch"].groupby("sig_preprocessing_grp").mean()
dataset_contact["coherence"] = dataset["coherence"].groupby("sig_preprocessing_pair_grp").mean()
dataset_contact["coherence_norm"] = dataset["coherence_norm"].groupby("sig_preprocessing_pair_grp").mean()
dataset_contact["duration"] = dataset["duration"].groupby("sig_preprocessing_grp").mean()
dataset_contact["common_duration"] = dataset["common_duration"].groupby("sig_preprocessing_pair_grp").mean()
dataset_contact


<xarray.Dataset>
Dimensions:                     (Contact: 5456, f_interp: 94, spectral_analysis_function: 3, sig_preprocessing_grp: 3, Contact_pair: 27559,
                                 sig_preprocessing_pair_grp: 6)
Coordinates:
  * Contact                     (Contact) int32 0 1 2 3 4 5 6 7 8 9 10 11 12 13 ... 5455 5456 5457 5458 5459 5460 5461 5462 5463 5464 5465 5466 5467
  * f_interp                    (f_interp) float64 3.0 3.5 4.0 4.5 5.0 5.5 6.0 6.5 7.0 7.5 8.0 ... 45.0 45.5 46.0 46.5 47.0 47.5 48.0 48.5 49.0 49.5
  * spectral_analysis_function  (spectral_analysis_function) object 'pycwt.cwt' 'scipy.coherence' 'scipy.spectrogram'
    Condition                   (Contact) object 'Park' 'Control_vnmr' 'Park' 'Control_vnmr' 'Control_vnmr' ... '6ohda' '6ohda' '6ohda' '6ohda'
    CorticalState               (Contact) object 'Awake' 'Awake' 'Awake' 'Awake' ... 'Anesthetized' 'Anesthetized' 'Anesthetized' 'Anesthetized'
    Session                     (Contact) object 'HS#1' 'HS#100' 'HS#100' 'HS#1000' 'HS#1002' ... 'RSa03#17' 'RSa03#17' 'RSa03#17' 'RSa03#17'
    FullStructure               (Contact) object 'STN_DLOR' 'STN_VMNR' 'STN_DLOR' 'STN_VMNR' 'STN_VMNR' 'STN_VMNR' ... 'STR' 'STR' 'STR' 'STR' 'STR'
    Subject                     (Contact) object 'Unknown' 'Unknown' 'Unknown' 'Unknown' 'Unknown' 'Unknown' ... 'L28' 'L28' 'L28' 'L28' 'L28' 'L28'
    Species                     (Contact) object 'Human' 'Human' 'Human' 'Human' 'Human' 'Human' 'Human' ... 'Rat' 'Rat' 'Rat' 'Rat' 'Rat' 'Rat' 'Rat'
    Structure                   (Contact) object 'STN' 'STN' 'STN' 'STN' 'STN' 'STN' 'STN' 'STN' ... 'STR' 'STR' 'STR' 'STR' 'STR' 'STR' 'STR' 'STR'
    Healthy                     (Contact) int8 0 1 0 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
    Date                        (Contact) datetime64[ns] 2009-01-14 2009-01-29 2009-01-29 2009-06-24 ... 2007-03-09 2007-03-09 2007-03-09 2007-03-09
    group_index                 (Contact) object ('Human', 'STN', 0) ('Human', 'STN', 1) ('Human', 'STN', 0) ... ('Rat', 'STR', 0) ('Rat', 'STR', 0)
    is_scipy_freq               (f_interp) bool True False True False True False True False True ... False True False True False True False True False
    Contact_grp                 (Contact) object ('Human', 'STN', 0) ('Human', 'STN', 1) ('Human', 'STN', 0) ... ('Rat', 'STR', 0) ('Rat', 'STR', 0)
  * sig_preprocessing_grp       (sig_preprocessing_grp) object MultiIndex
  * sig_type                    (sig_preprocessing_grp) object 'bua' 'lfp' 'spike_times'
  * Contact_pair                (Contact_pair) object MultiIndex
  * Contact_1                   (Contact_pair) int32 139 140 144 145 147 148 150 151 157 158 163 ... 5467 5467 5467 5467 5467 5467 5467 5467 5467 5467
  * Contact_2                   (Contact_pair) int32 140 139 145 144 148 147 151 150 158 157 164 ... 5457 5458 5459 5460 5461 5462 5463 5464 5465 5466
    Condition_1                 (Contact_pair) object 'Park' 'Park' 'Park' 'Park' 'Park' 'Park' ... '6ohda' '6ohda' '6ohda' '6ohda' '6ohda' '6ohda'
    CorticalState_1             (Contact_pair) object 'Awake' 'Awake' 'Awake' 'Awake' ... 'Anesthetized' 'Anesthetized' 'Anesthetized' 'Anesthetized'
    Session_1                   (Contact_pair) object 'HS#1246' 'HS#1246' 'HS#1250' 'HS#1250' ... 'RSa03#17' 'RSa03#17' 'RSa03#17' 'RSa03#17'
    FullStructure_1             (Contact_pair) object 'STN_DLOR' 'STN_DLOR' 'STN_DLOR' 'STN_DLOR' 'STN_DLOR' ... 'STR' 'STR' 'STR' 'STR' 'STR'
    Subject_1                   (Contact_pair) object 'Unknown' 'Unknown' 'Unknown' 'Unknown' 'Unknown' 'Unknown' ... 'L28' 'L28' 'L28' 'L28' 'L28'
    Species_1                   (Contact_pair) object 'Human' 'Human' 'Human' 'Human' 'Human' 'Human' 'Human' ... 'Rat' 'Rat' 'Rat' 'Rat' 'Rat' 'Rat'
    Structure_1                 (Contact_pair) object 'STN' 'STN' 'STN' 'STN' 'STN' 'STN' 'STN' 'STN' ... 'STR' 'STR' 'STR' 'STR' 'STR' 'STR' 'STR'
    Healthy_1

# Stats

## Creating bands

In [9]:
bands = xr.Dataset()
bands["band_start"] = xr.DataArray([4, 8, 15, 30], dims="band")
bands["band_end"] = xr.DataArray([8, 15, 30, 49], dims="band")
bands["band"] = ("band", ["before_beta", "low_beta", "high_beta", "after_beta"])
bands = bands.set_coords(["band_start", "band_end"])
dataset_contact = xr.merge([dataset_contact, bands])
dataset_contact

<xarray.Dataset>
Dimensions:                     (Contact: 5456, f_interp: 94, spectral_analysis_function: 3, sig_preprocessing_grp: 3, Contact_pair: 27559,
                                 sig_preprocessing_pair_grp: 6, band: 4)
Coordinates:
  * Contact                     (Contact) int32 0 1 2 3 4 5 6 7 8 9 10 11 12 13 ... 5455 5456 5457 5458 5459 5460 5461 5462 5463 5464 5465 5466 5467
  * f_interp                    (f_interp) float64 3.0 3.5 4.0 4.5 5.0 5.5 6.0 6.5 7.0 7.5 8.0 ... 45.0 45.5 46.0 46.5 47.0 47.5 48.0 48.5 49.0 49.5
  * spectral_analysis_function  (spectral_analysis_function) object 'pycwt.cwt' 'scipy.coherence' 'scipy.spectrogram'
    Condition                   (Contact) object 'Park' 'Control_vnmr' 'Park' 'Control_vnmr' 'Control_vnmr' ... '6ohda' '6ohda' '6ohda' '6ohda'
    CorticalState               (Contact) object 'Awake' 'Awake' 'Awake' 'Awake' ... 'Anesthetized' 'Anesthetized' 'Anesthetized' 'Anesthetized'
    Session                     (Contact) object 'HS#1' 'HS#100' 'HS#100' 'HS#1000' 'HS#1002' ... 'RSa03#17' 'RSa03#17' 'RSa03#17' 'RSa03#17'
    FullStructure               (Contact) object 'STN_DLOR' 'STN_VMNR' 'STN_DLOR' 'STN_VMNR' 'STN_VMNR' 'STN_VMNR' ... 'STR' 'STR' 'STR' 'STR' 'STR'
    Subject                     (Contact) object 'Unknown' 'Unknown' 'Unknown' 'Unknown' 'Unknown' 'Unknown' ... 'L28' 'L28' 'L28' 'L28' 'L28' 'L28'
    Species                     (Contact) object 'Human' 'Human' 'Human' 'Human' 'Human' 'Human' 'Human' ... 'Rat' 'Rat' 'Rat' 'Rat' 'Rat' 'Rat' 'Rat'
    Structure                   (Contact) object 'STN' 'STN' 'STN' 'STN' 'STN' 'STN' 'STN' 'STN' ... 'STR' 'STR' 'STR' 'STR' 'STR' 'STR' 'STR' 'STR'
    Healthy                     (Contact) int8 0 1 0 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
    Date                        (Contact) datetime64[ns] 2009-01-14 2009-01-29 2009-01-29 2009-06-24 ... 2007-03-09 2007-03-09 2007-03-09 2007-03-09
    group_index                 (Contact) object ('Human', 'STN', 0) ('Human', 'STN', 1) ('Human', 'STN', 0) ... ('Rat', 'STR', 0) ('Rat', 'STR', 0)
    is_scipy_freq               (f_interp) bool True False True False True False True False True ... False True False True False True False True False
    Contact_grp                 (Contact) object ('Human', 'STN', 0) ('Human', 'STN', 1) ('Human', 'STN', 0) ... ('Rat', 'STR', 0) ('Rat', 'STR', 0)
  * sig_preprocessing_grp       (sig_preprocessing_grp) object MultiIndex
  * sig_type                    (sig_preprocessing_grp) object 'bua' 'lfp' 'spike_times'
  * Contact_pair                (Contact_pair) object MultiIndex
  * Contact_1                   (Contact_pair) int32 139 140 144 145 147 148 150 151 157 158 163 ... 5467 5467 5467 5467 5467 5467 5467 5467 5467 5467
  * Contact_2                   (Contact_pair) int32 140 139 145 144 148 147 151 150 158 157 164 ... 5457 5458 5459 5460 5461 5462 5463 5464 5465 5466
    Condition_1                 (Contact_pair) object 'Park' 'Park' 'Park' 'Park' 'Park' 'Park' ... '6ohda' '6ohda' '6ohda' '6ohda' '6ohda' '6ohda'
    CorticalState_1             (Contact_pair) object 'Awake' 'Awake' 'Awake' 'Awake' ... 'Anesthetized' 'Anesthetized' 'Anesthetized' 'Anesthetized'
    Session_1                   (Contact_pair) object 'HS#1246' 'HS#1246' 'HS#1250' 'HS#1250' ... 'RSa03#17' 'RSa03#17' 'RSa03#17' 'RSa03#17'
    FullStructure_1             (Contact_pair) object 'STN_DLOR' 'STN_DLOR' 'STN_DLOR' 'STN_DLOR' 'STN_DLOR' ... 'STR' 'STR' 'STR' 'STR' 'STR'
    Subject_1                   (Contact_pair) object 'Unknown' 'Unknown' 'Unknown' 'Unknown' 'Unknown' 'Unknown' ... 'L28' 'L28' 'L28' 'L28' 'L28'
    Species_1                   (Contact_pair) object 'Human' 'Human' 'Human' 'Human' 'Human' 'Human' 'Human' ... 'Rat' 'Rat' 'Rat' 'Rat' 'Rat' 'Rat'
    Structure_1                 (Contact_pair) object 'STN' 'STN' 'STN' 'STN' 'STN' 'STN' 'STN' 'STN' ... 'STR' 'STR' 'STR' 'STR' 'STR' 'STR' 'STR'
    

In [10]:
dataset_contact["pwelch_band"] = dataset_contact["pwelch"].where(
    (dataset_contact["f_interp"] >= dataset_contact["band_start"]) & (dataset_contact["f_interp"] <= dataset_contact["band_end"])
).mean("f_interp")
dataset_contact

<xarray.Dataset>
Dimensions:                     (Contact: 5456, f_interp: 94, spectral_analysis_function: 3, sig_preprocessing_grp: 3, Contact_pair: 27559,
                                 sig_preprocessing_pair_grp: 6, band: 4)
Coordinates:
  * Contact                     (Contact) int32 0 1 2 3 4 5 6 7 8 9 10 11 12 13 ... 5455 5456 5457 5458 5459 5460 5461 5462 5463 5464 5465 5466 5467
  * f_interp                    (f_interp) float64 3.0 3.5 4.0 4.5 5.0 5.5 6.0 6.5 7.0 7.5 8.0 ... 45.0 45.5 46.0 46.5 47.0 47.5 48.0 48.5 49.0 49.5
  * spectral_analysis_function  (spectral_analysis_function) object 'pycwt.cwt' 'scipy.coherence' 'scipy.spectrogram'
    Condition                   (Contact) object 'Park' 'Control_vnmr' 'Park' 'Control_vnmr' 'Control_vnmr' ... '6ohda' '6ohda' '6ohda' '6ohda'
    CorticalState               (Contact) object 'Awake' 'Awake' 'Awake' 'Awake' ... 'Anesthetized' 'Anesthetized' 'Anesthetized' 'Anesthetized'
    Session                     (Contact) object 'HS#1' 'HS#100' 'HS#100' 'HS#1000' 'HS#1002' ... 'RSa03#17' 'RSa03#17' 'RSa03#17' 'RSa03#17'
    FullStructure               (Contact) object 'STN_DLOR' 'STN_VMNR' 'STN_DLOR' 'STN_VMNR' 'STN_VMNR' 'STN_VMNR' ... 'STR' 'STR' 'STR' 'STR' 'STR'
    Subject                     (Contact) object 'Unknown' 'Unknown' 'Unknown' 'Unknown' 'Unknown' 'Unknown' ... 'L28' 'L28' 'L28' 'L28' 'L28' 'L28'
    Species                     (Contact) object 'Human' 'Human' 'Human' 'Human' 'Human' 'Human' 'Human' ... 'Rat' 'Rat' 'Rat' 'Rat' 'Rat' 'Rat' 'Rat'
    Structure                   (Contact) object 'STN' 'STN' 'STN' 'STN' 'STN' 'STN' 'STN' 'STN' ... 'STR' 'STR' 'STR' 'STR' 'STR' 'STR' 'STR' 'STR'
    Healthy                     (Contact) int8 0 1 0 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
    Date                        (Contact) datetime64[ns] 2009-01-14 2009-01-29 2009-01-29 2009-06-24 ... 2007-03-09 2007-03-09 2007-03-09 2007-03-09
    group_index                 (Contact) object ('Human', 'STN', 0) ('Human', 'STN', 1) ('Human', 'STN', 0) ... ('Rat', 'STR', 0) ('Rat', 'STR', 0)
    is_scipy_freq               (f_interp) bool True False True False True False True False True ... False True False True False True False True False
    Contact_grp                 (Contact) object ('Human', 'STN', 0) ('Human', 'STN', 1) ('Human', 'STN', 0) ... ('Rat', 'STR', 0) ('Rat', 'STR', 0)
  * sig_preprocessing_grp       (sig_preprocessing_grp) object MultiIndex
  * sig_type                    (sig_preprocessing_grp) object 'bua' 'lfp' 'spike_times'
  * Contact_pair                (Contact_pair) object MultiIndex
  * Contact_1                   (Contact_pair) int32 139 140 144 145 147 148 150 151 157 158 163 ... 5467 5467 5467 5467 5467 5467 5467 5467 5467 5467
  * Contact_2                   (Contact_pair) int32 140 139 145 144 148 147 151 150 158 157 164 ... 5457 5458 5459 5460 5461 5462 5463 5464 5465 5466
    Condition_1                 (Contact_pair) object 'Park' 'Park' 'Park' 'Park' 'Park' 'Park' ... '6ohda' '6ohda' '6ohda' '6ohda' '6ohda' '6ohda'
    CorticalState_1             (Contact_pair) object 'Awake' 'Awake' 'Awake' 'Awake' ... 'Anesthetized' 'Anesthetized' 'Anesthetized' 'Anesthetized'
    Session_1                   (Contact_pair) object 'HS#1246' 'HS#1246' 'HS#1250' 'HS#1250' ... 'RSa03#17' 'RSa03#17' 'RSa03#17' 'RSa03#17'
    FullStructure_1             (Contact_pair) object 'STN_DLOR' 'STN_DLOR' 'STN_DLOR' 'STN_DLOR' 'STN_DLOR' ... 'STR' 'STR' 'STR' 'STR' 'STR'
    Subject_1                   (Contact_pair) object 'Unknown' 'Unknown' 'Unknown' 'Unknown' 'Unknown' 'Unknown' ... 'L28' 'L28' 'L28' 'L28' 'L28'
    Species_1                   (Contact_pair) object 'Human' 'Human' 'Human' 'Human' 'Human' 'Human' 'Human' ... 'Rat' 'Rat' 'Rat' 'Rat' 'Rat' 'Rat'
    Structure_1                 (Contact_pair) object 'STN' 'STN' 'STN' 'STN' 'STN' 'STN' 'STN' 'STN' ... 'STR' 'STR' 'STR' 'STR' 'STR' 'STR' 'STR'
    

## Bootstrapping

In [88]:
import concurrent
bootstrap_dataset = xr.Dataset()
n_boostrap = 100
pd.set_option('display.float_format', lambda x: f'{x:.2e}')
# bootstrap_dataset = bootstrap_dataset.assign_coords(significance=[0.05, 10**(-2), 10**(-3), 10**(-6)])

In [106]:
import concurrent
def bootstrap(f, arr: xr.DataArray, sample_dim, n_resamples=10000,  bt_dist="bt_dist", vectorize=False, executor_ctr=None):
    n = arr.sizes[sample_dim]
    if not executor_ctr is None:
        import concurrent
        futures = {}
        executor = executor_ctr().__enter__()
        

    if not vectorize:
        res=xr.Dataset()
        for i in tqdm.tqdm(np.arange(n_resamples), disable=False):
            choices = np.random.choice(np.arange(n), size=n, replace=True)
            sample = arr.isel({sample_dim:choices})
            if executor_ctr is None:
                res[i] = f(sample)
            else:
                futures[i] = executor.submit(f, sample)
        if not executor_ctr is None:
            for i, f in enumerate(concurrent.futures.as_completed(futures.values())):
                res[i] = f.result()
            executor.__exit__()
        return res.to_array(dim=bt_dist)
    else:
        if vectorize is True:
            vectorize=n_resamples
        all_res = []
        for i in range(int(np.ceil(n_resamples/vectorize))):
            samples = np.arange(i*vectorize, min((i+1)* vectorize, n_resamples))
            choices = np.random.choice(np.arange(n), size=len(samples)*arr.size, replace=True).reshape(arr.shape+(len(samples),))
            choices = xr.DataArray(data=choices, dims=[d if d != sample_dim else "bt_sample" for d in arr.dims] + [bt_dist], coords={bt_dist: samples, "bt_sample":np.arange(n)})
            sample: xr.DataArray = arr.isel({sample_dim:choices}, drop=True)
            sample = sample.drop([sample_dim])
            sample = sample.rename(bt_sample = sample_dim)
            if executor_ctr is None:
                all_res.append(f(sample))
            else:
                futures[i] = executor.submit(f, sample)
        if not executor_ctr is None:
            for f in concurrent.futures.as_completed(futures.values()):
                all_res.append(f.result())
            executor.__exit__()
        final= xr.concat(all_res, dim=bt_dist)
        # print(final)
        return final
    
def progress_map(self, f, *args, **kwargs):
    import tqdm
    bar = tqdm.notebook.tqdm(total=len(self.groups))
    def new_f(*a, **kw):
        bar.update(1)
        return f(*a, **kw)
    res = self.map(new_f, *args, **kwargs)
    bar.close()
    return res

xr.core.groupby.DataArrayGroupBy.progress_map = progress_map

def get_p_value(a):
    def pvalue(x):
        if np.isnan(x).any():
            return np.nan, np.nan
        nover = (x > 0).sum()
        nbelow = (x < 0).sum()
        if nover > nbelow:
            return "+", 1 - nover/x.size
        else:
            return "-", 1 - nbelow/x.size
    ret = xr.Dataset()
    ret["dir"], ret["pvalue"] = xr.apply_ufunc(pvalue,a,
        input_core_dims=[["bt_dist"]], output_core_dims=[[], []], output_dtypes=[object, float],  vectorize=True)
    return ret
    
def get_confidence_interval(a, q=[0.05, 0.5, 0.95]):
    return a.quantile(q, dim="bt_dist")



### Pwelch

In [108]:

bootstrap_dataset["pwelch_dist"] = dataset_contact["pwelch_band"].sel(spectral_analysis_function="scipy.spectrogram", sig_type="bua").groupby("Contact_grp").progress_map(
    lambda a: bootstrap(lambda ar: ar.mean(dim="Contact"), a, sample_dim = "Contact", n_resamples=n_boostrap, vectorize=1000, 
                        # executor_ctr=lambda: concurrent.futures.ThreadPoolExecutor(10)
    )).unstack()

get_confidence_interval(bootstrap_dataset["pwelch_dist"].sel(Healthy=0) - bootstrap_dataset["pwelch_dist"].sel(Healthy=1)).to_dataframe().unstack("quantile")

  0%|          | 0/16 [00:00<?, ?it/s]

[2024-03-12 13:01:24,898] WARNING  @py.warnings warnings.py:109 : /home/julien/miniconda3/envs/electrophy/lib/python3.11/site-packages/numpy/lib/nanfunctions.py:1563: RuntimeWarning: All-NaN slice encountered
  return function_base._ureduce(a,



band_start          band_end           \
quantile                        5.00e-02 9.50e-01 5.00e-02 9.50e-01   
band        Species Structure                                         
before_beta Human   GPe                4        4        8        8   
                    STN                4        4        8        8   
                    STR                4        4        8        8   
            Monkey  GPe                4        4        8        8   
                    STN                4        4        8        8   
                    STR                4        4        8        8   
            Rat     GPe                4        4        8        8   
                    STN                4        4        8        8   
                    STR                4        4        8        8   
low_beta    Human   GPe                8        8       15       15   
                    STN                8        8       15       15   
                    STR                8        8       15       15   
            Monkey  GPe                8        8       15       15   
                    STN                8        8       15       15   
                    STR                8        8       15       15   
            Rat     GPe                8        8       15       15   
                    STN                8        8       15       15   
                    STR                8        8       15       15   
high_beta   Human   GPe               15       15       30       30   
                    STN               15       15       30       30   
                    STR               15       15       30       30   
            Monkey  GPe               15       15       30       30   
                    STN               15       15       30       30   
                    STR               15       15       30       30   
            Rat     GPe               15       15       30       30   
                    STN               15       15       30       30   
                    STR               15       15       30       30   
after_beta  Human   GPe               30       30       49       49   
                    STN               30       30       49       49   
                    STR               30       30       49       49   
            Monkey  GPe               30       30       49       49   
                    STN               30       30       49       49   
                    STR               30       30       49       49   
            Rat     GPe               30       30       49       49   
                    STN               30       30       49       49   
                    STR               30       30       49       49   

                              pwelch_dist            
quantile                         5.00e-02  9.50e-01  
band        Species Structure                        
before_beta Human   GPe               NaN       NaN  
                    STN          2.97e-03  3.68e-03  
                    STR               NaN       NaN  
            Monkey  GPe         -5.83e-04  1.51e-03  
                    STN          2.15e-03  3.61e-03  
                    STR          6.00e-04  2.22e-03  
            Rat     GPe          9.27e-04  1.41e-03  
                    STN         -8.54e-03 -3.44e-03  
                    STR          6.69e-05  5.34e-04  
low_beta    Human   GPe               NaN       NaN  
                    STN          3.20e-03  3.74e-03  
                    STR               NaN       NaN  
            Monkey  GPe          1.05e-03  2.33e-03  
                    STN          7.99e-03  1.05e-02  
                    STR          1.28e-03  2.42e-03  
            Rat     GPe          2.12e-03  2.59e-03  
                    STN         -2.12e-03 -6.66e-05  
                    STR          1.49e-04  6.47e-04  
high_beta   Human   GPe               NaN       NaN  
                    STN          2.49e-03  2.74e-03  
                  

In [105]:
def compute_max_f(a):
    # print(a)
    # from time import sleep
    ret = a.sel(f_interp=slice(8, 34)).mean(dim="Contact").idxmax("f_interp")
    # print(ret)
    # sleep(1)
    return ret

bootstrap_dataset["pwelch_max_f_dist"] = dataset_contact["pwelch"].sel(spectral_analysis_function="scipy.spectrogram", sig_type="bua").groupby("Contact_grp").progress_map(
    lambda a: bootstrap(compute_max_f, a, sample_dim = "Contact", n_resamples=n_boostrap, vectorize=1000, 
                        # executor_ctr=lambda: concurrent.futures.ThreadPoolExecutor(10)
    )).unstack()

bootstrap_dataset["pwelch_max_f_dist"].mean("bt_dist").to_dataframe()

  0%|          | 0/16 [00:00<?, ?it/s]

spectral_analysis_function sig_preprocessing_grp  \
Species Structure Healthy                                                    
Human   GPe       0                scipy.spectrogram                (bua,)   
                  1                scipy.spectrogram                (bua,)   
        STN       0                scipy.spectrogram                (bua,)   
                  1                scipy.spectrogram                (bua,)   
        STR       0                scipy.spectrogram                (bua,)   
                  1                scipy.spectrogram                (bua,)   
Monkey  GPe       0                scipy.spectrogram                (bua,)   
                  1                scipy.spectrogram                (bua,)   
        STN       0                scipy.spectrogram                (bua,)   
                  1                scipy.spectrogram                (bua,)   
        STR       0                scipy.spectrogram                (bua,)   
                  1                scipy.spectrogram                (bua,)   
Rat     GPe       0                scipy.spectrogram                (bua,)   
                  1                scipy.spectrogram                (bua,)   
        STN       0                scipy.spectrogram                (bua,)   
                  1                scipy.spectrogram                (bua,)   
        STR       0                scipy.spectrogram                (bua,)   
                  1                scipy.spectrogram                (bua,)   

                          sig_type  pwelch_max_f_dist  
Species Structure Healthy                              
Human   GPe       0            bua           1.62e+01  
                  1            bua                NaN  
        STN       0            bua           1.13e+01  
                  1            bua           2.54e+01  
        STR       0            bua           1.86e+01  
                  1            bua                NaN  
Monkey  GPe       0            bua           1.01e+01  
                  1            bua           8.26e+00  
        STN       0            bua           1.07e+01  
                  1            bua           8.74e+00  
        STR       0            bua           9.57e+00  
                  1            bua           8.66e+00  
Rat     GPe       0            bua           2.15e+01  
                  1            bua           3.32e+01  
        STN       0            bua           1.72e+01  
                  1            bua           8.26e+00  
        STR       0            bua           1.31e+01  
                  1            bua           1.14e+01

In [43]:
import scipy.stats




def mbootstrap(h, p):
    if np.count_nonzero(~np.isnan(h)) <1 or np.count_nonzero(~np.isnan(p))<1:
        return np.full(n_boostrap , np.nan)
    h = h[~np.isnan(h)]
    p = p[~np.isnan(p)]
    r = scipy.stats.bootstrap([p, h], lambda x, y, axis: np.nanmean(x, axis=axis)-np.nanmean(y, axis=axis), n_resamples=n_boostrap, vectorized=True)
    return r.bootstrap_distribution 



def test(x):
    healthy =  x.where(dataset_contact["Healthy"], drop=True)
    park = x.where(~dataset_contact["Healthy"], drop=True)
    r = xr.apply_ufunc(mbootstrap, healthy, park, input_core_dims=[["Contact"]]*2, exclude_dims={"Contact"}, output_core_dims=[["bt_dist"]], vectorize=True)
    return r



data = dataset_contact[["pwelch_band"]].sel(spectral_analysis_function="scipy.spectrogram", sig_type="bua")
data["Structure_grp"] = xr.apply_ufunc(lambda x: str(x[0:-1]), data["Contact_grp"], output_core_dims=[[]], vectorize=True)
data = data.set_coords("Structure_grp")
bootstrap_dataset["pwelch_val"]  = data["pwelch_band"].groupby("Structure_grp").map(test)
bootstrap_dataset

<xarray.Dataset>
Dimensions:                     (band: 4, Structure_grp: 9, bt_dist: 10000)
Coordinates:
  * band                        (band) <U11 'before_beta' 'low_beta' 'high_beta' 'after_beta'
    spectral_analysis_function  <U17 'scipy.spectrogram'
    sig_preprocessing_grp       object ('bua',)
    sig_type                    <U3 'bua'
    band_start                  (band) int64 4 8 15 30
    band_end                    (band) int64 8 15 30 49
  * Structure_grp               (Structure_grp) object "('Human', 'GPe')" "('Human', 'STN')" "('Human', 'STR')" ... "('Rat', 'STN')" "('Rat', 'STR')"
Dimensions without coordinates: bt_dist
Data variables:
    pwelch_val                  (Structure_grp, band, bt_dist) float64 nan nan nan nan nan nan ... 0.00014 9.088e-05 0.0001518 0.0004203 0.0003338

In [148]:
import scipy.stats

class stupid:
    def __init__(self, a):
        self.a = a

def mbootstrap(x, y, f):
    if np.count_nonzero(~np.isnan(x)) <1 or np.count_nonzero(~np.isnan(y))<1:
        return np.full(n_boostrap , np.nan)
    x = x[~np.isnan(x).all(axis=1)]
    y = y[~np.isnan(y).all(axis=1)]

    res = np.empty(n_boostrap)
    for i in tqdm.tqdm(np.arange(n_boostrap), disable=True):
        sample1_index = np.random.choice(np.arange(x.shape[0]), size=x.shape[0])
        sample1 = x[sample1_index]
        sample2_index = np.random.choice(np.arange(y.shape[0]), size=y.shape[0])
        sample2 = y[sample2_index]
        freqs1 = sample1.mean(axis=0)
        freqs2 = sample2.mean(axis=0)
        f1 =f[np.argmax(freqs1)]
        f2 = f[np.argmax(freqs2)]
        res[i] = f1 - f2
    return res



data = dataset_contact[["pwelch"]].sel(spectral_analysis_function="scipy.spectrogram", sig_type="bua", f_interp=slice(8, 34)).where(data["Healthy"]<1, drop=True)
data["Structure_grp"] = xr.apply_ufunc(lambda x: str(x[0:-1]), data["Contact_grp"], output_core_dims=[[]], vectorize=True)
data = data.set_coords("Structure_grp")
res = {}
for grpname1, grp1 in tqdm.tqdm(data["pwelch"].groupby("Structure_grp")):
    
    res[grpname1] = xr.Dataset()
    for grpname2, grp2 in data["pwelch"].groupby("Structure_grp"):
        # tmp1 = xr.apply_ufunc(lambda a: stupid(a) if not np.isnan(a).any() else np.nan, grp1, input_core_dims=[["f_interp"]], output_core_dims=[[]], vectorize=True)
        # tmp2 = xr.apply_ufunc(lambda a: stupid(a) if not np.isnan(a).any() else np.nan, grp2, input_core_dims=[["f_interp"]], output_core_dims=[[]], vectorize=True)
        r = xr.apply_ufunc(mbootstrap, grp1, grp2, data["f_interp"], input_core_dims=[["Contact", "f_interp"]]*2 +[["f_interp"]], exclude_dims={"Contact"}, output_core_dims=[["bt_dist"]], vectorize=True)
        res[grpname1][grpname2] = r

tmp=xr.Dataset()
for n1, arr in tqdm.tqdm(res.items()):
    tmp[n1] = arr.to_array(dim="grp2")

res = tmp.to_array(dim="grp1")
bootstrap_dataset["pwelch_f"]  =  res
bootstrap_dataset

100%|██████████| 9/9 [00:00<00:00, 916.21it/s]


<xarray.Dataset>
Dimensions:                     (spectral_analysis_function: 3, sig_preprocessing_grp: 3, band: 4, Structure_grp: 9, bt_dist: 10000, f_interp: 94,
                                 variable: 9, grp1: 9, grp2: 9)
Coordinates:
  * spectral_analysis_function  (spectral_analysis_function) object 'pycwt.cwt' 'scipy.coherence' 'scipy.spectrogram'
  * sig_preprocessing_grp       (sig_preprocessing_grp) object MultiIndex
  * sig_type                    (sig_preprocessing_grp) object 'bua' 'lfp' 'spike_times'
  * band                        (band) <U11 'before_beta' 'low_beta' 'high_beta' 'after_beta'
    band_start                  (band) int64 4 8 15 30
    band_end                    (band) int64 8 15 30 49
  * Structure_grp               (Structure_grp) object "('Human', 'GPe')" "('Human', 'STN')" "('Human', 'STR')" ... "('Rat', 'STN')" "('Rat', 'STR')"
  * f_interp                    (f_interp) float64 3.0 3.5 4.0 4.5 5.0 5.5 6.0 6.5 7.0 7.5 8.0 ... 45.0 45.5 46.0 46.5 47.0 47.5 48.0 48.5 49.0 49.5
    is_scipy_freq               (f_interp) bool True False True False True False True False True ... False True False True False True False True False
  * variable                    (variable) object "('Human', 'GPe')" "('Human', 'STN')" "('Human', 'STR')" ... "('Rat', 'STN')" "('Rat', 'STR')"
  * grp2                        (grp2) object "('Human', 'GPe')" "('Human', 'STN')" "('Human', 'STR')" ... "('Rat', 'STN')" "('Rat', 'STR')"
  * grp1                        (grp1) object "('Human', 'GPe')" "('Human', 'STN')" "('Human', 'STR')" ... "('Rat', 'STN')" "('Rat', 'STR')"
Dimensions without coordinates: bt_dist
Data variables:
    pwelch_dist                 (Structure_grp, band, bt_dist) float64 nan nan nan nan nan nan ... 0.0001565 9.927e-05 8.852e-06 0.0001092 0.0001864
    pwelch_f                    (grp1, grp2, bt_dist) float64 -1.0 13.0 -19.0 1.0 13.0 -5.0 0.0 -1.0 1.0 ... -11.0 -11.0 0.0 0.0 -11.0 0.0 0.0 0.0 0.0

### Computing significance levels

In [149]:
def pvalue(x):
    if np.isnan(x).any():
        return np.nan, np.nan
    nover = (x > 0).sum()
    nbelow = (x < 0).sum()
    if nover > nbelow:
        return "+", 1 - nover/x.size
    else:
        return "-", 1 - nbelow/x.size
    
significance_dataset = xr.Dataset()
for v in bootstrap_dataset.data_vars:
    significance_dataset[[f"{v}_direction", f"{v}_pvalue"]] = xr.apply_ufunc(pvalue,bootstrap_dataset[v],
        input_core_dims=[["bt_dist"]], output_core_dims=[[], []], output_dtypes=[object, float],  vectorize=True)

In [150]:
pd.set_option('display.float_format', lambda x: f'{x:.2e}')
print(significance_dataset[["pwelch_f_direction", "pwelch_f_pvalue"]].to_dataframe().to_string())

                                    pwelch_f_direction  pwelch_f_pvalue
grp1              grp2                                                 
('Human', 'GPe')  ('Human', 'GPe')                   +         6.19e-01
                  ('Human', 'STN')                   +         4.69e-01
                  ('Human', 'STR')                   -         2.85e-01
                  ('Monkey', 'GPe')                  +         4.17e-01
                  ('Monkey', 'STN')                  +         4.57e-01
                  ('Monkey', 'STR')                  +         3.86e-01
                  ('Rat', 'GPe')                     -         2.86e-01
                  ('Rat', 'STN')                     +         4.99e-01
                  ('Rat', 'STR')                     +         3.08e-01
('Human', 'STN')  ('Human', 'GPe')                   -         4.71e-01
                  ('Human', 'STN')                   +         7.20e-01
                  ('Human', 'STR')                   -         1

In [57]:
import scipy.stats
def mbootstrap(x, y):
    print(x.shape, y.shape)

    # x = x[~np.isnan(x)]
    # y = y[~np.isnan(y)]
    def get_max(x, y, axis):
        print(x.shape, y.shape, axis)
        raise Exception("Stop")
    r = scipy.stats.bootstrap((x,y), get_max, axis=[0, 1])
    
    # raise Exception("Stop")
    if np.isnan(r.confidence_interval[1]):
        res=  np.nan
    else:
        if r.confidence_interval[1] < 0:
            res = 1.0
        elif r.confidence_interval[0] > 0:
            res = -1.0
        else: 
            res = 0.0
    # print(res)
    return res

data = dataset_contact[["pwelch"]].sel(spectral_analysis_function="scipy.spectrogram", sig_type="bua").sel(f_interp = slice(8, 34))
data = data.where(data["Healthy"]<1, drop=True)

data["Structure_grp"] = xr.apply_ufunc(lambda x: str(x[1:]), dataset_contact["Contact_grp"], output_core_dims=[[]], vectorize=True)
data = data.set_coords("Structure_grp")

def test(x):
    # print(x)
    vals = ["Monkey", "Human", "Rat"]
    r = []
    for i in range(3):
        for j in range(i+1, 3):
            r.append(xr.apply_ufunc(
                mbootstrap, x.where(x["Species"] == vals[i], drop=True), x.where(x["Species"] == vals[j], drop=True), 
                input_core_dims=[["Contact", "f_interp"]]*2, 
                exclude_dims={"Contact"}, vectorize=True).expand_dims(Sp1=[vals[i]], Sp2 = vals[j]))
    # raise Exception("Stop")
    return xr.merge(r)

res = data["pwelch"].groupby("Structure_grp").map(test)
print(res)
res.sel(spectral_analysis_function="scipy.spectrogram", sig_type="bua").to_dataframe(name="boostrap significative")

(104, 53) (30, 53)


TypeError: int() argument must be a string, a bytes-like object or a real number, not 'list'

# Creating grouped dataset

In [ ]:
grouped_dataset = xr.Dataset()

grouped_dataset["pwelch"] = dataset["pwelch"].groupby("sig_preprocessing_grp").mean().groupby("Contact_grp").mean()
grouped_dataset["n_contacts"] = dataset["pwelch"].groupby("sig_preprocessing_grp").mean().groupby("Contact_grp").count("Contact").sel(f_interp=4)
grouped_dataset["duration"] = dataset["duration"].groupby("sig_preprocessing_grp").mean().groupby("Contact_grp").mean("Contact")
grouped_dataset["coherence"] = dataset["coherence"].groupby("sig_preprocessing_pair_grp").mean().groupby("Contact_pair_grp").mean()
grouped_dataset["coherence_norm"] = dataset["coherence_norm"].groupby("sig_preprocessing_pair_grp").mean().groupby("Contact_pair_grp").mean()
grouped_dataset["n_contact_pairs"] = dataset["coherence"].groupby("sig_preprocessing_pair_grp").mean().groupby("Contact_pair_grp").count("Contact_pair").sel(f_interp=4)
grouped_dataset["common_duration"] = dataset["common_duration"].groupby("sig_preprocessing_pair_grp").mean().groupby("Contact_pair_grp").mean("Contact_pair")
grouped_dataset = grouped_dataset.set_coords(["n_contact_pairs", "n_contacts"])

grouped_dataset["coherence_phase"] = xr.apply_ufunc(np.angle, grouped_dataset["coherence"])
grouped_dataset["coherence_validity"] = np.abs(grouped_dataset["coherence"])/grouped_dataset["coherence_norm"]
grouped_dataset["f_max"] = grouped_dataset["coherence_norm"].sel(f_interp=slice(7, 40)).idxmax("f_interp")
grouped_dataset

<xarray.Dataset>
Dimensions:                     (f_interp: 94, spectral_analysis_function: 3, sig_preprocessing_grp: 3, Contact_grp: 16,
                                 sig_preprocessing_pair_grp: 6, Contact_pair_grp: 35)
Coordinates:
  * f_interp                    (f_interp) float64 3.0 3.5 4.0 4.5 5.0 5.5 6.0 6.5 7.0 7.5 8.0 ... 45.0 45.5 46.0 46.5 47.0 47.5 48.0 48.5 49.0 49.5
  * spectral_analysis_function  (spectral_analysis_function) object 'pycwt.cwt' 'scipy.coherence' 'scipy.spectrogram'
    is_scipy_freq               (f_interp) bool True False True False True False True False True ... False True False True False True False True False
  * sig_preprocessing_grp       (sig_preprocessing_grp) object MultiIndex
  * sig_type                    (sig_preprocessing_grp) object 'bua' 'lfp' 'spike_times'
  * Contact_grp                 (Contact_grp) object MultiIndex
  * Species                     (Contact_grp) object 'Human' 'Human' 'Human' 'Human' 'Monkey' 'Monkey' ... 'Rat' 'Rat' 'Rat' 'Rat' 'Rat' 'Rat'
  * Structure                   (Contact_grp) object 'STN' 'STN' 'GPe' 'STR' 'GPe' 'STR' 'STN' 'STN' 'GPe' 'STR' 'GPe' 'STN' 'STR' 'GPe' 'STN' 'STR'
  * Healthy                     (Contact_grp) int64 0 1 0 0 1 1 1 0 0 0 1 1 1 0 0 0
    n_contacts                  (spectral_analysis_function, Contact_grp, sig_preprocessing_grp) int64 1538 0 822 1958 0 954 ... 18 18 16 314 314 45
  * sig_preprocessing_pair_grp  (sig_preprocessing_pair_grp) object MultiIndex
  * sig_type_1                  (sig_preprocessing_pair_grp) object 'bua' 'bua' 'bua' 'lfp' 'lfp' 'spike_times'
  * sig_type_2                  (sig_preprocessing_pair_grp) object 'bua' 'lfp' 'spike_times' 'lfp' 'spike_times' 'spike_times'
  * Contact_pair_grp            (Contact_pair_grp) object MultiIndex
  * Species_1                   (Contact_pair_grp) object 'Human' 'Human' 'Human' 'Human' 'Human' 'Human' ... 'Rat' 'Rat' 'Rat' 'Rat' 'Rat' 'Rat'
  * Structure_1                 (Contact_pair_grp) object 'STN' 'GPe' 'STN' 'GPe' 'STN' 'STR' 'STR' ... 'STN' 'STN' 'STR' 'GPe' 'STR' 'STN' 'STR'
  * Healthy_1                   (Contact_pair_grp) float64 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 1.0 0.0 0.0 ... 1.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0
  * Species_2                   (Contact_pair_grp) object 'Human' 'Human' 'Human' 'Human' 'Human' 'Human' ... 'Rat' 'Rat' 'Rat' 'Rat' 'Rat' 'Rat'
  * Structure_2                 (Contact_pair_grp) object 'STN' 'STN' 'GPe' 'GPe' 'STR' 'STN' 'STR' ... 'GPe' 'STN' 'STR' 'STR' 'GPe' 'STR' 'STN'
  * Healthy_2                   (Contact_pair_grp) float64 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 1.0 0.0 0.0 ... 1.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0
    n_contact_pairs             (spectral_analysis_function, sig_preprocessing_pair_grp, Contact_pair_grp) int64 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0
Data variables:
    pwelch                      (spectral_analysis_function, Contact_grp, sig_preprocessing_grp, f_interp) float64 nan nan 1.9 ... 0.007605 0.00759
    duration                    (sig_preprocessing_grp, Contact_grp) float64 20.63 16.69 51.08 16.5 1.396e+03 ... 95.62 89.55 99.43 97.45 96.37
    coherence                   (spectral_analysis_function, sig_preprocessing_pair_grp, Contact_pair_grp, f_interp) complex128 (nan+0j) ... (nan+0j)
    coherence_norm              (spectral_analysis_function, sig_preprocessing_pair_grp, Contact_pair_grp, f_interp) float64 nan nan nan ... nan nan
    common_duration             (sig_preprocessing_pair_grp, Contact_pair_grp) float64 25.95 22.89 22.89 31.05 14.71 ... 94.78 93.41 92.06 nan nan
    coherence_phase             (spectral_analysis_function, sig_preprocessing_pair_grp, Contact_pair_grp, f_interp) float64 nan nan nan ... nan nan
    coherence_validity          (spectral_analysis_function, sig_preprocessing_pair_grp, Contact_pair_grp, f_interp) float64 nan nan nan ... nan nan
    f_max                       (spectral_analysis_function, sig_preprocessing_pair_grp, Contact_pair_grp) float64 n

# Adding the column of interest

In [ ]:
print(dataset)
print(grouped_dataset)


<xarray.Dataset>
Dimensions:                     (Contact: 5456, sig_preprocessing: 5, sig_preprocessing_pair: 15, Contact_pair: 27559, f_interp: 94,
                                 spectral_analysis_function: 3)
Coordinates:
  * Contact                     (Contact) int32 0 1 2 3 4 5 6 7 8 9 10 11 12 13 ... 5455 5456 5457 5458 5459 5460 5461 5462 5463 5464 5465 5466 5467
  * sig_preprocessing           (sig_preprocessing) object 'bua' 'lfp' 'neuron_0' 'neuron_1' 'neuron_2'
  * sig_preprocessing_pair      (sig_preprocessing_pair) object MultiIndex
  * sig_preprocessing_1         (sig_preprocessing_pair) object 'bua' 'bua' 'bua' 'bua' 'bua' ... 'neuron_0' 'neuron_1' 'neuron_1' 'neuron_2'
  * sig_preprocessing_2         (sig_preprocessing_pair) object 'bua' 'lfp' 'neuron_0' 'neuron_1' ... 'neuron_2' 'neuron_1' 'neuron_2' 'neuron_2'
  * Contact_pair                (Contact_pair) object MultiIndex
  * Contact_1                   (Contact_pair) int32 139 140 144 145 147 148 150 151 157 158

In [ ]:
def f(x: np.ndarray, coord, dim):
    other = dataset[dim].to_numpy()
    print(x.shape, other.shape)
    
    positions = np.where(coord[:, None] == other[None, :])[0]
    res = np.apply_along_axis(lambda x: x[positions], -1, x)
    return res

f_max_tmp = xr.apply_ufunc(lambda x, y: f(x, y, "Contact_pair_grp"), grouped_dataset["f_max"], grouped_dataset["Contact_pair_grp"], input_core_dims=[["Contact_pair_grp"], ["Contact_pair_grp"]], output_core_dims=[["Contact_pair"]])
f_max_tmp = xr.apply_ufunc(lambda x, y: f(x, y, "sig_preprocessing_pair_grp"), f_max_tmp, grouped_dataset["sig_preprocessing_pair_grp"], input_core_dims=[["sig_preprocessing_pair_grp"], ["sig_preprocessing_pair_grp"]], output_core_dims=[["sig_preprocessing_pair"]])
dataset["f_max_coherence"] = f_max_tmp
print(dataset)

(3, 6, 35) (27559,)
(3, 27559, 6) (15,)
<xarray.Dataset>
Dimensions:                     (Contact: 5456, sig_preprocessing: 5, sig_preprocessing_pair: 15, Contact_pair: 27559, f_interp: 94,
                                 spectral_analysis_function: 3)
Coordinates:
  * Contact                     (Contact) int32 0 1 2 3 4 5 6 7 8 9 10 11 12 13 ... 5455 5456 5457 5458 5459 5460 5461 5462 5463 5464 5465 5466 5467
  * sig_preprocessing           (sig_preprocessing) object 'bua' 'lfp' 'neuron_0' 'neuron_1' 'neuron_2'
  * sig_preprocessing_pair      (sig_preprocessing_pair) object MultiIndex
  * sig_preprocessing_1         (sig_preprocessing_pair) object 'bua' 'bua' 'bua' 'bua' 'bua' ... 'neuron_0' 'neuron_1' 'neuron_1' 'neuron_2'
  * sig_preprocessing_2         (sig_preprocessing_pair) object 'bua' 'lfp' 'neuron_0' 'neuron_1' ... 'neuron_2' 'neuron_1' 'neuron_2' 'neuron_2'
  * Contact_pair                (Contact_pair) object MultiIndex
  * Contact_1                   (Contact_pair) int32

In [ ]:

tmp_group = dataset["coherence"].sel(f_interp=dataset["f_max_coherence"].fillna(4)).groupby("sig_preprocessing_pair_grp").mean().groupby("Contact_pair_grp")


In [ ]:

phase_bins = np.linspace(-np.pi, np.pi, 180)

def mk_phase_info(x: xr.DataArray):
    m = x*np.exp(-1j * xr.DataArray(phase_bins, dims=["phase_bins"], coords=dict(phase_bins=phase_bins)))
    angle = xr.apply_ufunc(np.angle, m)
    dot = np.real(m)
    res = dot.where(abs(angle) <np.pi/5).sum("Contact_pair")
    return res

grouped_dataset["coherence_phase_bins"] = tmp_group.map(mk_phase_info)
print(grouped_dataset["coherence_phase_bins"])

<xarray.DataArray 'coherence_phase_bins' (spectral_analysis_function: 3, sig_preprocessing_pair_grp: 6, Contact_pair_grp: 35, phase_bins: 180)>
array([[[[ 0.        ,  0.        ,  0.        , ...,  0.        ,
           0.        ,  0.        ],
         [ 0.        ,  0.        ,  0.        , ...,  0.        ,
           0.        ,  0.        ],
         [ 0.        ,  0.        ,  0.        , ...,  0.        ,
           0.        ,  0.        ],
         ...,
         [ 0.        ,  0.        ,  0.        , ...,  0.        ,
           0.        ,  0.        ],
         [ 0.        ,  0.        ,  0.        , ...,  0.        ,
           0.        ,  0.        ],
         [ 0.        ,  0.        ,  0.        , ...,  0.        ,
           0.        ,  0.        ]],

        [[ 0.        ,  0.        ,  0.        , ...,  0.        ,
           0.        ,  0.        ],
         [ 0.        ,  0.        ,  0.        , ...,  0.        ,
           0.        ,  0.        ],
        

In [ ]:
from xarray_helper import normalize


# Post handling

In [ ]:


# print(grouped_dataset["coherence_phase_bins"].sel(f_interp=grouped_dataset["f_max"].fillna(grouped_dataset["f_interp"].min())))
# exit()

selected = (grouped_dataset["f_interp"] > grouped_dataset["f_max"]-2) & (grouped_dataset["f_interp"] < grouped_dataset["f_max"]+2) & grouped_dataset["is_scipy_freq"]
grouped_dataset["coherence_phase"] = grouped_dataset["coherence_phase"].where(selected)

for col in group_cols:
    if grouped_dataset[f"{col}_1"].equals(grouped_dataset[f"{col}_2"]):
        grouped_dataset[col+"(common)"] = grouped_dataset[f"{col}_1"]
        grouped_dataset=grouped_dataset.set_coords(col+"(common)")
    if dataset_contact[f"{col}_1"].equals(dataset_contact[f"{col}_2"]):
        dataset_contact[col+"(common)"] = dataset_contact[f"{col}_1"]
        dataset_contact=dataset_contact.set_coords(col+"(common)")

print(grouped_dataset)

<xarray.Dataset>
Dimensions:                     (f_interp: 94, spectral_analysis_function: 3, sig_preprocessing_grp: 3, Contact_grp: 16,
                                 sig_preprocessing_pair_grp: 6, Contact_pair_grp: 35, phase_bins: 180)
Coordinates:
  * f_interp                    (f_interp) float64 3.0 3.5 4.0 4.5 5.0 5.5 6.0 6.5 7.0 7.5 8.0 ... 45.0 45.5 46.0 46.5 47.0 47.5 48.0 48.5 49.0 49.5
  * spectral_analysis_function  (spectral_analysis_function) object 'pycwt.cwt' 'scipy.coherence' 'scipy.spectrogram'
    is_scipy_freq               (f_interp) bool True False True False True False True False True ... False True False True False True False True False
  * sig_preprocessing_grp       (sig_preprocessing_grp) object MultiIndex
  * sig_type                    (sig_preprocessing_grp) object 'bua' 'lfp' 'spike_times'
  * Contact_grp                 (Contact_grp) object MultiIndex
  * Species                     (Contact_grp) object 'Human' 'Human' 'Human' 'Human' 'Monkey' 'Monkey

In [ ]:
print(grouped_dataset)

<xarray.Dataset>
Dimensions:                     (f_interp: 94, spectral_analysis_function: 3, sig_preprocessing_grp: 3, Contact_grp: 16,
                                 sig_preprocessing_pair_grp: 6, Contact_pair_grp: 35, phase_bins: 180)
Coordinates:
  * f_interp                    (f_interp) float64 3.0 3.5 4.0 4.5 5.0 5.5 6.0 6.5 7.0 7.5 8.0 ... 45.0 45.5 46.0 46.5 47.0 47.5 48.0 48.5 49.0 49.5
  * spectral_analysis_function  (spectral_analysis_function) object 'pycwt.cwt' 'scipy.coherence' 'scipy.spectrogram'
    is_scipy_freq               (f_interp) bool True False True False True False True False True ... False True False True False True False True False
  * sig_preprocessing_grp       (sig_preprocessing_grp) object MultiIndex
  * sig_type                    (sig_preprocessing_grp) object 'bua' 'lfp' 'spike_times'
  * Contact_grp                 (Contact_grp) object MultiIndex
  * Species                     (Contact_grp) object 'Human' 'Human' 'Human' 'Human' 'Monkey' 'Monkey

# Drawing coherence phase

In [ ]:
dataset

<xarray.Dataset>
Dimensions:                     (Contact: 5456, sig_preprocessing: 5, sig_preprocessing_pair: 15, Contact_pair: 27559, f_interp: 94,
                                 spectral_analysis_function: 3)
Coordinates:
  * Contact                     (Contact) int32 0 1 2 3 4 5 6 7 8 9 10 11 12 13 ... 5455 5456 5457 5458 5459 5460 5461 5462 5463 5464 5465 5466 5467
  * sig_preprocessing           (sig_preprocessing) object 'bua' 'lfp' 'neuron_0' 'neuron_1' 'neuron_2'
  * sig_preprocessing_pair      (sig_preprocessing_pair) object MultiIndex
  * sig_preprocessing_1         (sig_preprocessing_pair) object 'bua' 'bua' 'bua' 'bua' 'bua' ... 'neuron_0' 'neuron_1' 'neuron_1' 'neuron_2'
  * sig_preprocessing_2         (sig_preprocessing_pair) object 'bua' 'lfp' 'neuron_0' 'neuron_1' ... 'neuron_2' 'neuron_1' 'neuron_2' 'neuron_2'
  * Contact_pair                (Contact_pair) object MultiIndex
  * Contact_1                   (Contact_pair) int32 139 140 144 145 147 148 150 151 157 158 163 ... 5467 5467 5467 5467 5467 5467 5467 5467 5467 5467
  * Contact_2                   (Contact_pair) int32 140 139 145 144 148 147 151 150 158 157 164 ... 5457 5458 5459 5460 5461 5462 5463 5464 5465 5466
  * f_interp                    (f_interp) float64 3.0 3.5 4.0 4.5 5.0 5.5 6.0 6.5 7.0 7.5 8.0 ... 45.0 45.5 46.0 46.5 47.0 47.5 48.0 48.5 49.0 49.5
  * spectral_analysis_function  (spectral_analysis_function) object 'pycwt.cwt' 'scipy.coherence' 'scipy.spectrogram'
    Condition                   (Contact) object 'Park' 'Control_vnmr' 'Park' 'Control_vnmr' 'Control_vnmr' ... '6ohda' '6ohda' '6ohda' '6ohda'
    CorticalState               (Contact) object 'Awake' 'Awake' 'Awake' 'Awake' ... 'Anesthetized' 'Anesthetized' 'Anesthetized' 'Anesthetized'
    Session                     (Contact) object 'HS#1' 'HS#100' 'HS#100' 'HS#1000' 'HS#1002' ... 'RSa03#17' 'RSa03#17' 'RSa03#17' 'RSa03#17'
    FullStructure               (Contact) object 'STN_DLOR' 'STN_VMNR' 'STN_DLOR' 'STN_VMNR' 'STN_VMNR' 'STN_VMNR' ... 'STR' 'STR' 'STR' 'STR' 'STR'
    Subject                     (Contact) object 'Unknown' 'Unknown' 'Unknown' 'Unknown' 'Unknown' 'Unknown' ... 'L28' 'L28' 'L28' 'L28' 'L28' 'L28'
    Species                     (Contact) object 'Human' 'Human' 'Human' 'Human' 'Human' 'Human' 'Human' ... 'Rat' 'Rat' 'Rat' 'Rat' 'Rat' 'Rat' 'Rat'
    has_entry                   (Contact, sig_preprocessing) bool True False True False False True False ... True False True True True False False
    Structure                   (Contact) object 'STN' 'STN' 'STN' 'STN' 'STN' 'STN' 'STN' 'STN' ... 'STR' 'STR' 'STR' 'STR' 'STR' 'STR' 'STR' 'STR'
    Healthy                     (Contact) int8 0 1 0 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
    Date                        (Contact) datetime64[ns] 2009-01-14 2009-01-29 2009-01-29 2009-06-24 ... 2007-03-09 2007-03-09 2007-03-09 2007-03-09
    sig_type                    (sig_preprocessing) object 'bua' 'lfp' 'spike_times' 'spike_times' 'spike_times'
    group_index                 (Contact) object ('Human', 'STN', 0) ('Human', 'STN', 1) ('Human', 'STN', 0) ... ('Rat', 'STR', 0) ('Rat', 'STR', 0)
    Condition_1                 (Contact_pair) object 'Park' 'Park' 'Park' 'Park' 'Park' 'Park' ... '6ohda' '6ohda' '6ohda' '6ohda' '6ohda' '6ohda'
    CorticalState_1             (Contact_pair) object 'Awake' 'Awake' 'Awake' 'Awake' ... 'Anesthetized' 'Anesthetized' 'Anesthetized' 'Anesthetized'
    Session_1                   (Contact_pair) object 'HS#1246' 'HS#1246' 'HS#1250' 'HS#1250' ... 'RSa03#17' 'RSa03#17' 'RSa03#17' 'RSa03#17'
    FullStructure_1             (Contact_pair) object 'STN_DLOR' 'STN_DLOR' 'STN_DLOR' 'STN_DLOR' 'STN_DLOR' ... 'STR' 'STR' 'STR' 'STR' 'STR'
    Subject_1                   (Contact_pair) object 'Unknown' 'Unknown' 'Unknown' 'Unknown' 'Unknown' 'Unknown' ... 'L28' 'L28' 'L28' 'L28' 'L28'
    Species_1                   (Contact_pair) object 

In [ ]:

n_contacts: xr.DataArray = grouped_dataset[["n_contacts", "duration"]]
n_contacts = n_contacts.sel(sig_preprocessing_grp="bua", drop=True).isel(spectral_analysis_function=0).unstack().squeeze()
res = n_contacts.to_dataframe()[["n_contacts", "duration"]]
res["n_data"] = res["n_contacts"] * res["duration"]/1000
res = res.apply(np.round).dropna().astype(int)
res = res.sort_values("n_data")
res["n_data"]="~"+ res["n_data"].astype(str) + " 000"
res

# ["n_contacts"].iloc[:, 0:1].sort_values("n_contacts")

n_contacts  duration    n_data
Species Structure Healthy                                
Human   STR       0                57        17    ~1 000
        GPe       0                30        51    ~2 000
Rat     STN       0                18        96    ~2 000
                  1                21        96    ~2 000
        STR       1               149        99   ~15 000
        GPe       1               199        99   ~20 000
        STR       0               314       100   ~31 000
Human   STN       0              1538        21   ~32 000
                  1              1958        17   ~33 000
Rat     GPe       0               402       100   ~40 000
Monkey  STN       0               111       520   ~58 000
        GPe       0               104       584   ~61 000
        STR       0               127       596   ~76 000
        STN       1               101      1053  ~106 000
        STR       1               146      1039  ~152 000
        GPe       1               181      1396  ~253 000

In [ ]:
n_contacts_pairs: xr.DataArray = grouped_dataset[["n_contact_pairs", "common_duration"]].unstack("sig_preprocessing_pair_grp")

n_contacts_pairs = n_contacts_pairs.sel(sig_type_1="bua", sig_type_2="bua", drop=True).squeeze()
n_contacts_pairs = n_contacts_pairs.isel(spectral_analysis_function=1)
res = n_contacts_pairs.rename(Species_1="Species", Healthy_1="Healthy").to_dataframe()[["n_contact_pairs", "common_duration"]]
res = res.reset_index(["Species_2", "Healthy_2"], drop=True).reset_index()
res = res[res["Structure_1"] == res["Structure_2"]]
res = res.set_index(["Species", "Healthy", "Structure_1", "Structure_2"])
res = res[res["n_contact_pairs"] >10]

res["n_data"] = res["n_contact_pairs"] * res["common_duration"]/1000
res = res.apply(np.round).dropna().astype(int)
res = res.sort_values("n_data")
res["n_data"]="~"+ res["n_data"].astype(str) + " 000"
res
# n_contacts_pairs = n_contacts_pairs[n_contacts_pairs["n_contact_pairs"]>10].reset_index()
# n_contacts_pairs["Species"] = n_contacts_pairs["Species_1"]
# n_contacts_pairs["Healthy"] = n_contacts_pairs["Healthy_1"]
# n_contacts_pairs = n_contacts_pairs.drop(columns=["Species_1", "Species_2", "Healthy_1", "Healthy_2"])
# n_contacts_pairs = n_contacts_pairs.set_index(["Species",	"Healthy", "Structure_1", "Structure_2"]).sort_values("n_contact_pairs")
# n_contacts_pairs.to_dataframe()["n_contact_pairs"].iloc[:, 0:1]


n_contact_pairs  common_duration  \
Species Healthy Structure_1 Structure_2                                     
Human   0.0     STN         STN                       72               26   
Rat     0.0     STN         STN                       26               95   
        1.0     STN         STN                       35               97   
Monkey  0.0     STN         STN                       14              209   
        1.0     STN         STN                       18              640   
        0.0     STR         STR                       52              337   
                GPe         GPe                       78              407   
        1.0     STR         STR                       53              611   
Rat     1.0     STR         STR                      698              100   
                GPe         GPe                     1286               99   
Monkey  1.0     GPe         GPe                      133              974   
Rat     0.0     STR         STR                     2116              100   
                GPe         GPe                     3037              100   

                                           n_data  
Species Healthy Structure_1 Structure_2            
Human   0.0     STN         STN            ~2 000  
Rat     0.0     STN         STN            ~2 000  
        1.0     STN         STN            ~3 000  
Monkey  0.0     STN         STN            ~3 000  
        1.0     STN         STN           ~12 000  
        0.0     STR         STR           ~18 000  
                GPe         GPe           ~32 000  
        1.0     STR         STR           ~32 000  
Rat     1.0     STR         STR           ~69 000  
                GPe         GPe          ~128 000  
Monkey  1.0     GPe         GPe          ~130 000  
Rat     0.0     STR         STR          ~212 000  
                GPe         GPe          ~303 000

In [ ]:
data = grouped_dataset["pwelch"].where(grouped_dataset["n_contacts"] > 0).sel(spectral_analysis_function="scipy.spectrogram").unstack().sel(sig_type="bua").rename(f_interp="f")
toolbox.FigurePlot(
    data = data.to_dataframe(name="pwelch").dropna(axis="index", subset="pwelch"),
    figures="spectral_analysis_function", col="Structure", row="sig_type", sharey=False, margin_titles=True, fig_title="mean_pwelch_structure_{spectral_analysis_function}",
).map(sns.lineplot, x="f", y="pwelch", hue="Species", style="Healthy", hue_order=species_order, style_order=condition_order).add_legend()
plt.show()

In [ ]:
data = grouped_dataset["pwelch"].where(grouped_dataset["n_contacts"] > 0).sel(spectral_analysis_function="scipy.spectrogram").unstack().sel(sig_type="bua")
toolbox.FigurePlot(
    data = data.to_dataframe(name="pwelch").dropna(axis="index", subset="pwelch"),
    figures="spectral_analysis_function", col="Species", row="sig_type", sharey=False, margin_titles=True, fig_title="mean_pwelch_species_{spectral_analysis_function}",
).map(sns.lineplot, x="f_interp", y="pwelch", hue="Structure", style="Healthy", 
      hue_order=structure_order, style_order=condition_order, palette=sns.color_palette("dark", n_colors=len(structure_order))).add_legend()
plt.show()

In [ ]:
data = grouped_dataset[["coherence_norm", "coherence_phase", "coherence_validity", "f_max"]].where(grouped_dataset["n_contact_pairs"] > 10)
data = data.sel(spectral_analysis_function="scipy.coherence").sel(sig_type_1="bua").sel(sig_type_2="bua").rename(f_interp="f")
data = data.where(data["Structure_1"]==data["Structure_2"]).rename(Structure_1="Structure").rename({"Species(common)":"Species", "Healthy(common)":"Healthy"})
data = data.where(data["Healthy"]==0)
data["f_max_amp"] = data["coherence_norm"].sel(f= data["f_max"].fillna(4))
(toolbox.FigurePlot(
    data = data.to_dataframe().dropna(axis="index", subset="coherence_norm"),
    figures=["spectral_analysis_function", "sig_type_1", "sig_type_2"], col="Structure",
    sharey=False, margin_titles=True, fig_title="mean_coherence_structure_{spectral_analysis_function}, {sig_type_1}, {sig_type_2}", 
    col_order=structure_order,
).map(sns.scatterplot, x="f_max", y="f_max_amp", c="red").map(sns.lineplot, x="f", y="coherence_norm", hue="Species", hue_order=species_order)

).add_legend()
plt.show()

In [ ]:
data = grouped_dataset[["coherence_norm", "coherence_phase", "coherence_validity", "f_max"]].where(grouped_dataset["n_contact_pairs"] > 10)
data = data.sel(spectral_analysis_function="scipy.coherence").sel(sig_type_1="bua").sel(sig_type_2="bua").rename(f_interp="f")
data = data.where(data["Structure_1"]!=data["Structure_2"]).rename({"Species(common)":"Species", "Healthy(common)":"Healthy"})
data = data.where(data["Healthy"]==0)
data["f_max_amp"] = data["coherence_norm"].sel(f= data["f_max"].fillna(4))
data["Structure"] = data["Structure_1"].astype(str).astype(object) +", " +  data["Structure_2"].astype(str).astype(object)
(toolbox.FigurePlot(
    data = data.to_dataframe().dropna(axis="index", subset="coherence_norm"),
    figures=["spectral_analysis_function", "sig_type_1", "sig_type_2"],
    sharey=False, margin_titles=True, fig_title="mean_coherence_structure_{spectral_analysis_function}, {sig_type_1}, {sig_type_2}", 
).map(sns.scatterplot, x="f_max", y="f_max_amp", c="red").map(sns.lineplot, x="f", y="coherence_norm", hue="Structure")

).add_legend()
plt.show()

In [ ]:
grouped_dataset["coherence_phase_bins"] = grouped_dataset["coherence_phase_bins"]/grouped_dataset["coherence_phase_bins"].max("phase_bins")
data = (
    grouped_dataset["coherence_phase_bins"]
# .where(grouped_dataset["sig_type_1"]=="bua")

    .where(grouped_dataset["n_contact_pairs"] > 10)
    .where(grouped_dataset["Healthy(common)"] <0.5)
)
data = data.sel(spectral_analysis_function="scipy.coherence").sel(sig_type_1="bua").sel(sig_type_2="bua")
data = data.where(data["Structure_1"]!=data["Structure_2"]).rename({"Species(common)":"Species", "Healthy(common)":"Healthy"})
data["Structure"] = data["Structure_1"].astype(str).astype(object) +", " +  data["Structure_2"].astype(str).astype(object)
data = data.to_dataframe().dropna(axis="index", subset="coherence_phase_bins")
(toolbox.FigurePlot(
    data = data,
    figures=["spectral_analysis_function", "sig_type_1", "sig_type_2"],
    sharey=False, margin_titles=True, fig_title="coherence_phase_bins_structure_{spectral_analysis_function}, {sig_type_1}, {sig_type_2}", 
    subplot_kws=dict(projection='polar'), despine=False,
).map(sns.lineplot, x="phase_bins", y="coherence_phase_bins", hue="Structure")
.add_legend().set(yticks=[]))
# .maximize().save_pdf(f"{cache_path}Figures/coherence_phase_bins_structure.pdf")
plt.show()

In [ ]:
grouped_dataset["coherence_phase_bins"] = grouped_dataset["coherence_phase_bins"]/grouped_dataset["coherence_phase_bins"].max("phase_bins")
data = (
    grouped_dataset["coherence_phase_bins"]
# .where(grouped_dataset["sig_type_1"]=="bua")

    .where(grouped_dataset["n_contact_pairs"] > 10)
    .where(grouped_dataset["Healthy(common)"] <0.5)
)
data = data.sel(spectral_analysis_function="scipy.coherence").sel(sig_type_1="bua").sel(sig_type_2="bua")
data = data.where(data["Structure_1"]==data["Structure_2"]).rename(Structure_1="Structure").rename({"Species(common)":"Species", "Healthy(common)":"Healthy"})
print(data)
data = data.to_dataframe().dropna(axis="index", subset="coherence_phase_bins")
print(data)
(toolbox.FigurePlot(
    data = data,
    figures=["spectral_analysis_function", "sig_type_1", "sig_type_2"], col="Structure",
    sharey=False, margin_titles=True, fig_title="coherence_phase_bins_structure_{spectral_analysis_function}, {sig_type_1}, {sig_type_2}", 
    col_order=structure_order,
    subplot_kws=dict(projection='polar'), despine=False,
).map(sns.lineplot, x="phase_bins", y="coherence_phase_bins", hue="Species", style="Healthy", hue_order=species_order, style_order=condition_order)
.add_legend().set(yticks=[]))
# .maximize().save_pdf(f"{cache_path}Figures/coherence_phase_bins_structure.pdf")
plt.show()

<xarray.DataArray 'coherence_phase_bins' (Contact_pair_grp: 35, phase_bins: 180)>
array([[0.03800728, 0.03784132, 0.03740026, ..., 0.03819858, 0.03812641,
        0.03800728],
       [       nan,        nan,        nan, ...,        nan,        nan,
               nan],
       [       nan,        nan,        nan, ...,        nan,        nan,
               nan],
       ...,
       [       nan,        nan,        nan, ...,        nan,        nan,
               nan],
       [       nan,        nan,        nan, ...,        nan,        nan,
               nan],
       [       nan,        nan,        nan, ...,        nan,        nan,
               nan]])
Coordinates:
    spectral_analysis_function  <U15 'scipy.coherence'
    sig_type_2                  <U3 'bua'
  * Contact_pair_grp            (Contact_pair_grp) object MultiIndex
  * Species_1                   (Contact_pair_grp) object 'Human' 'Human' 'Human' 'Human' 'Human' 'Human' ... 'Rat' 'Rat' 'Rat' 'Rat' 'Rat' 'Rat'
  * Structure   